<a href="https://colab.research.google.com/github/weraines/nfl_model/blob/master/game_stats_initial_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv

url_base = 'https://www.teamrankings.com'

#Read in a csv with the various stats url endings
with open("urls.csv", 'r') as my_file:
    reader = csv.reader(my_file)
    my_list = list(reader)
    
#Create emply list to hold base stat urls
urls = []

for x in my_list:
    for y in x:
        urls.append(url_base+y)
        

#Create a list of dates to get stats by week
dates = ['?date=2018-09-11', '?date=2018-09-18', '?date=2018-09-25', 
         '?date=2018-10-02', '?date=2018-10-09', '?date=2018-10-16', 
         '?date=2018-10-23', '?date=2018-10-30', '?date=2018-11-06', 
         '?date=2018-11-06',]

data = [] #Holds dowloaded stat data
week = [] #Keeps track of game week
src = [] #Keeps track of the source url

#initializes week to week 2.  
#Week 1 stats are uaed to forecast week 2 games, etc.
w = 2

for date in dates:
    for u in urls:
        t_url = u+date
        website_url = requests.get(t_url)
        soup = BeautifulSoup(website_url.content, 'html.parser')
        table = soup.find_all('table')[0]
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])
            week.append(w)
            src.append(t_url)
    w += 1

In [0]:
columns = ['Rank', 'Team', '2018', 'Last 3', 'Last 1', 'Home', 'Away', 
           '2017']
df_stats = pd.DataFrame.from_records(data)

df_stats.columns = columns

df_stats['week'] = week
df_stats['source'] = src

In [0]:
stat_name = []
prefix = 'https://www.teamrankings.com/nfl/stat/'

for index, row in df_stats.iterrows():
    stat_name.append(row['source'][len(prefix):len(row['source'])-16])

df_stats['stat_name'] = stat_name

In [0]:
df_stats = df_stats[['Rank', 'Team', 'week', 'source', 'stat_name',
                     '2018', 'Last 3', 'Last 1', 'Home', 'Away', '2017',]]

df_stats = pd.melt(df_stats, id_vars=['Rank', 'Team', 'week', 'source', 
                                      'stat_name',])

In [0]:
df_2018 = df_stats[df_stats.variable == '2018']

In [0]:
df_2018.to_csv('2018stats.csv', encoding='utf-8')

In [0]:
df_elo = pd.read_csv(
    'https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv')
df_2018 = df_elo[df_elo.season == 2018]

In [13]:
df_2018

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
16007,2018-09-06,2018,0,NaN,PHI,ATL,1646.786330,1600.640440,0.654710,0.345290,1659.578148,1587.848622,18.0,12.0
16008,2018-09-09,2018,0,NaN,BAL,BUF,1535.270103,1502.216567,0.637480,0.362520,1561.692326,1475.794344,47.0,3.0
16009,2018-09-09,2018,0,NaN,IND,CIN,1406.904376,1474.615256,0.496099,0.503901,1382.279534,1499.240097,23.0,34.0
16010,2018-09-09,2018,0,NaN,CLE,PIT,1301.910164,1595.802933,0.211216,0.788784,1310.717644,1586.995454,21.0,21.0
16011,2018-09-09,2018,0,NaN,NE,HOU,1651.078663,1397.603393,0.862153,0.137847,1656.086591,1392.595466,27.0,20.0
16012,2018-09-09,2018,0,NaN,MIA,TEN,1449.870530,1495.841219,0.527358,0.472642,1469.358591,1476.353157,27.0,20.0
16013,2018-09-09,2018,0,NaN,NYG,JAX,1411.677494,1534.818404,0.417101,0.582899,1397.115446,1549.380453,15.0,20.0
16014,2018-09-09,2018,0,NaN,NO,TB,1584.490340,1469.481636,0.738119,0.261881,1549.163577,1504.808399,40.0,48.0
16015,2018-09-09,2018,0,NaN,MIN,SF,1602.219546,1469.474445,0.757370,0.242630,1612.002483,1459.691508,24.0,16.0
16016,2018-09-09,2018,0,NaN,LAC,KC,1544.922535,1570.892675,0.555933,0.444067,1517.779595,1598.035615,28.0,38.0


In [0]:
df_2018 = df_2018.dropna(subset = ['team1','score1'])

In [0]:
elo_teams = np.sort(df_2018.team1.unique()).tolist()

ksports_teams = np.sort(df_stats.Team.unique()).tolist()

team_abbrev = dict(zip(ksports_teams, elo_teams))

team_abbrev

In [0]:
df_stats['abbr'] = df_stats['Team'].map(team_abbrev)

In [0]:
df_stats = df_stats[['Rank', 'Team', 'abbr', 'week', 'source', 'stat_name', 
                     'variable', 'value',]]

In [0]:
df_2018.date = pd.to_datetime(df_2018.date)

In [0]:
win=[]
spread=[]

for index, row in df_2018.iterrows():
    if row['score1'] > row['score2']:
        win.append('team1')
    elif row['score1'] < row['score2']:
        win.append('team2')
    else:
        win.append('tie')
    spread.append(row['score1'] - row['score2'])

In [0]:
df_2018['win'] = win
df_2018['spread'] = spread
df_2018

In [0]:
df_2018.to_csv('elo_2018.csv', encoding='utf-8')

In [0]:
df_stats.info()

In [0]:
import datetime

week = []

for index, row in df_2018.iterrows():
    print(date)
    if row['date'] < datetime.date(2018,9,11):
        wk = 1
    elif row['date'] > datetime.date(2018,9,11) and row['date'] < datetime.date(2018,9,18):
        wk = 2
    elif row['date'] > datetime.date(2018,9,18) and row['date'] < datetime.date(2018,9,25):
        wk = 3
    elif row['date'] > datetime.date(2018,9,25) and row['date'] < datetime.date(2018,10,2):
        wk = 4
    elif row['date'] > datetime.date(2018,10,2) and row['date'] < datetime.date(2018,10,9):
        wk = 5
    elif row['date'] > datetime.date(2018,10,9) and row['date'] < datetime.date(2018,10,16):
        wk = 6
    elif row['date'] > datetime.date(2018,10,16) and row['date'] < datetime.date(2018,10,23):
        wk = 7
    else:
        wk = 8
    week.append(wk)

df_2018['week'] = week

In [0]:
df_2018.info()